In [ ]:
!pip install "matplotlib>=3.2,<=3.7.3"
!pip install "ydata-profiling>4.4,<4.5"
!pip install tensorflow

## Ideas

If adversary does not work, predict -> predict_proba to get more information

In [ ]:
import pandas as pd
import numpy as np
from pia_functions import data_train_test, get_distributed_adult_sets, generate_shadow_model_outputs

### Generate training data for shadow models

In [ ]:
n_shadow_models=200
distributions=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [ ]:
X_train, X_test, y_train, y_test, sensitive, sensitive_t = data_train_test()

In [ ]:
distributed_datasets = get_distributed_adult_sets(distributions=distributions)

### Train shadow models and generate output

In [ ]:
all_shadow_outputs = []
for ds in distributed_datasets:
    outputs = generate_shadow_model_outputs(ds, X_test, n_shadow_models=n_shadow_models)
    all_shadow_outputs.append(outputs)

### Save shadow model outputs to file

In [ ]:
adv_df = pd.DataFrame(np.array(np.concatenate((all_shadow_outputs))))
adv_df["y"] = np.concatenate(([np.repeat(d, n_shadow_models) for d in distributions]))
adv_df.to_csv("shadow_model_outputs.csv")

### Shadow model outputs as training data for adversary

In [ ]:
adv_df = pd.read_csv("shadow_model_outputs.csv")
adv_ddf_shuffled = adv_df.sample(frac=1, random_state=1).reset_index(drop=True)
adv_y = adv_ddf_shuffled["y"]
adv_X = adv_ddf_shuffled.drop(columns=["y"])

### Create adversary

In [69]:
from tensorflow import keras
adversary = keras.Sequential()
adversary.add(keras.Input(shape=(adv_X.shape[1],)))
adversary.add(keras.layers.Dense(8, activation='relu'))
adversary.add(keras.layers.Dense(4, activation='relu'))
adversary.add(keras.layers.Dense(1))

adversary.compile(optimizer='adam', loss=keras.losses.MeanSquaredError())

### Train adversary

In [70]:
adversary.fit(adv_X, adv_y, batch_size=32, epochs=10)

Epoch 1/10
57/57 [==============================] - 1s 4ms/step - loss: 8.5975
Epoch 2/10
57/57 [==============================] - 0s 4ms/step - loss: 0.3002
Epoch 3/10
57/57 [==============================] - 0s 4ms/step - loss: 0.0556
Epoch 4/10
57/57 [==============================] - 0s 4ms/step - loss: 0.0041
Epoch 5/10
57/57 [==============================] - 0s 4ms/step - loss: 0.0025
Epoch 6/10
57/57 [==============================] - 0s 4ms/step - loss: 0.0020
Epoch 7/10
57/57 [==============================] - 0s 4ms/step - loss: 0.0016
Epoch 8/10
57/57 [==============================] - 0s 4ms/step - loss: 0.0014
Epoch 9/10
57/57 [==============================] - 0s 4ms/step - loss: 0.0012
Epoch 10/10
57/57 [==============================] - 0s 4ms/step - loss: 0.0011


### Next steps
* Test adversary (create test set)
* Use adversary during training

### (Data set analysis)

In [ ]:
import ydata_profiling

ydata_profiling.ProfileReport(X_test)

In [ ]:
# RANDOM DS
random_dict = {}
for attr in ds.X_train:
    print(f'{round(X_train_pr[attr].min(),2)} {round(X_train_pr[attr].max(),2)}')
    random_attr = np.random.rand(random_length)*(X_train_pr[attr].max()+abs(X_train_pr[attr].min()))-abs(X_train_pr[attr].min())
    random_dict[attr] = random_attr

In [ ]:
random_ds = pd.DataFrame(random_dict)